# Synchronized tdmclient interactive session
## Graphics

This notebook illustrates the use of `tdmclient.notebook` with graphics provided by the `matplotlib` package.

As usual, make sure the latest version of tdmclient is installed (you can skip this step if you did it recently):

In [ ]:
%pip install --upgrade tdmclient

Import the required classes and connect to the robot. In addition to `tdmclient.notebook`, `matplotlib` provides support for graphics.

In [ ]:
import tdmclient.notebook
import matplotlib.pyplot as plt
await tdmclient.notebook.start()

To plot a sensor value, or any computed value, as a function of time, you can retrieve the values with events.

Let's begin with the example presented to illustrate the use of events. The program will run for 4 seconds. You can move your hand in front of robot.

In [ ]:
%%run_python --clear-event-data --wait

i = 0
timer_period[0] = 200

@onevent
def timer0():
    global i, prox_horizontal
    i += 1
    if i > 20:
        exit()
    emit("front", prox_horizontal[2])

Then we retrieve and plot the event data:

In [ ]:
%matplotlib inline
prox_front = get_event_data("front")
plt.plot(prox_front);

The horizontal scale shows the sample index, from 0 to 20 (the `_exit` event sent by the call to `exit()` is processed by the PC after the complete execution of `timer0()`; thus the program emits values for `i` from 1 to 21).

You may prefer to use a time scale. If the events are produced in a timer event at a known rate, the time can be computed in the notebook. But often it's more convenient to get the actual time on the robot by reading its clock. For that, we use the `ticks_50Hz()` function defined in the `clock` module, which returns a value incremented 50 times per second. Instead of counting samples, we stop when the clock reaches 4 seconds. Both `clock.ticks_50Hz()` and `clock.seconds()` are reset to 0 when the program starts or when `clock.reset()` is called. Here is a new version of the robot program:

In [ ]:
%%run_python --clear-event-data --wait

import clock

timer_period[0] = 200

@onevent
def timer0():
    global prox_horizontal
    if clock.seconds() >= 4:
        exit()
    emit("front", clock.ticks_50Hz(), prox_horizontal[2])

The events produced by `emit()` contain 2 values, the number of ticks and the front proximity sensor. We can extract them into `t` and `y` with list comprehensions, a compact way to manipulate list values. The time is converted to seconds as fractional number, something which cannot be done on the Thymio where all numbers are integers.

In [ ]:
%matplotlib inline
prox_front = get_event_data("front")
t = [data[0] / 50 for data in prox_front]
y = [data[1] for data in prox_front]
plt.plot(t, y);

### Live graphics

Support for animated graphics, where new data are displayed when there're available, depends on the version of Jupyter and the extensions which are installed. This section describes one way to update a figure in JupyterLab without any extension.

We modify the program and plot above to run continuously with a sliding time window of 10 seconds. The call to `exit()` is removed from the robot program, and we don't wait for the program to terminate.

In [ ]:
%%run_python --clear-event-data

import clock

timer_period[0] = 200

@onevent
def timer0():
    global prox_horizontal
    emit("front", clock.ticks_50Hz(), prox_horizontal[2])

The figure below displays the last 10 seconds of data in a figure which is updated everytime new events are received. For each event received, the first data value is the time in 1/50 second, and the remaining values are displayed as separate lines. Thus you can keep the same code with different robot programs, as long as you emit events with a unique name and a fixed number of values.

Click the stop button in the toolbar above to interrupt the kernel (the Python session which executes the notebook cells).

In [ ]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
%matplotlib inline

def on_event_data(event_name):

    def update_plot(t, y, time_span=10):
        clear_output(wait=True)
        plt.figure()

        if len(t) > 1:
            plt.plot(t, y)
            t_last = t[-1]
            plt.xlim(t_last - time_span, t_last)

        plt.grid(True)
        plt.show();

    data_list = get_event_data(event_name)
    t = [data[0] / 50 for data in data_list]
    y = [data[1:] for data in data_list]

    update_plot(t, y)

clear_event_data()
tdmclient.notebook.process_events(on_event_data)